## Example for callbacks + Memory profiling of verbs and workflow

In this example we show how to use the workflow callbacks to be able profile the memory usage of every verb

In [1]:
import json
import os
from pathlib import Path

import pandas as pd
from datashaper import MemoryProfile, Workflow

FIXTURES_PATH = "../../../schema/fixtures/workflow"
TABLE_STORE_PATH = "../../../schema/fixtures/workflow_inputs"
profile = None


tables: dict[str, pd.DataFrame] = {}
for file in os.listdir(TABLE_STORE_PATH):
    if file.endswith(".csv"):
        path = Path(TABLE_STORE_PATH) / file
        table = pd.read_csv(
            path,
        )
        tables[file.removesuffix(".csv")] = table


def get_verb_test_specs(root: str) -> list[str]:
    """Get the test specs for the verbs."""
    subfolders: list[str] = []
    for dirname, _, files in os.walk(root):
        if "workflow.json" in files:
            subfolders.append(dirname)
    return subfolders


async def test_verbs_schema_input(fixture_path: str) -> MemoryProfile:
    """Test the schema input of the verbs."""
    with (Path(fixture_path) / "workflow.json").open() as schema:
        workflow = Workflow(
            memory_profile=True,
            schema=json.load(schema),
            input_tables=tables,
        )
    result = await workflow.run()
    return result.memory_profile

In [2]:
for fixture_path in get_verb_test_specs(FIXTURES_PATH):
    profile = await test_verbs_schema_input(fixture_path)

/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/fold.py:22: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  output = output.stack(dropna=False).reset_index()
/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/fold.py:22: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  output = output.stack(dropna=False).reset_index()
/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/fold.py:22: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of panda

Test print verb
    ID       Name  Employees     US
0    1  Microsoft     160000   True
1    2      Apple     150000   True
..  ..        ...        ...    ...
3    4     Amazon    1250000   True
4    5    Samsung     270000  False


/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/convert.py:71: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  datetime_column = pd.to_datetime(column, errors="ignore")
/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/convert.py:71: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  datetime_column = pd.to_datetime(column, errors="ignore")
/home/andresmor/Projects/datashaper/python/datashaper/datashaper/engine/verbs/convert.py:64: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  column_numeric = cast(pd.Series, pd.to_numeric(column, errors="ignore"))
/home/andresmor/Projects/datashaper/pytho

In [3]:
profile.snapshot_stats

,mean,max,min,samples
all,0.030230,0.030230,0.030230,1.0
onehot,0.027885,0.027885,0.027885,1.0


In [4]:
profile.peak_stats

,mean,max,min,samples
all,0.045373,0.045373,0.045373,1.0
onehot,0.037442,0.037442,0.037442,1.0


In [5]:
profile.time_stats

,mean,max,min,samples
all,0.010951,0.010951,0.010951,1.0
onehot,0.009947,0.009947,0.009947,1.0


In [6]:
dive_deep = profile.detailed_view

In [7]:
dive_deep[
    (dive_deep["verb"] == "onehot")
    & (dive_deep["sample"] == 0)
    & (dive_deep["filename"].str.contains("onehot"))
]

,verb,size_diff,size,count_diff,count,filename,lineno,sample
204,onehot,112,112,2,2,/home/andresmor/Projects/datashaper/python/dat...,35,0
